In [1]:
from polygon import RESTClient
from datetime import datetime, date, timedelta
from tqdm import tqdm
import pandas as pd
import yfinance as yf
import sqlite3

connection = sqlite3.connect('data/bio/candles.db')

In [2]:
df = pd.read_csv('data/bio/finviz_exported.csv').drop(columns=['No.', 'P/E', 'Change', 'Volume'])
df

,Ticker,Company,Sector,Industry,Country,Market Cap,Price
0,LLY,Lilly(Eli) & Co,Healthcare,Drug Manufacturers - General,USA,822096.00,865.00
1,NVO,Novo Nordisk ADR,Healthcare,Biotechnology,Denmark,479498.72,143.63
2,UNH,Unitedhealth Group Inc,Healthcare,Healthcare Plans,USA,455588.10,495.00
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,USA,354094.84,147.13
4,MRK,Merck & Co Inc,Healthcare,Drug Manufacturers - General,USA,333039.20,131.49
...,...,...,...,...,...,...,...
1197,SYRA,Syra Health Corp,Healthcare,Medical Care Facilities,USA,NaN,1.52
1198,TLX,Telix Pharmaceuticals Limited,Healthcare,Biotechnology,Australia,NaN,NaN
1199,TEM,"Tempus AI, Inc.",Healthcare,Diagnostics & Research,USA,NaN,NaN
1200,NTHI,NeOnc Technologies Holdings Inc.,Healthcare,Biotechnology,USA,NaN,NaN


In [3]:
df = df[
    (df['Market Cap'] > 20) &
#    (df['Country'] == 'USA') &
    df['Industry'].isin([
        'Biotechnology',
        'Drug Manufacturers - General',
        'Diagnostics & Research',
        'Drug Manufacturers - Specialty & Generic'
    ])
]
df

,Ticker,Company,Sector,Industry,Country,Market Cap,Price
0,LLY,Lilly(Eli) & Co,Healthcare,Drug Manufacturers - General,USA,822096.00,865.00
1,NVO,Novo Nordisk ADR,Healthcare,Biotechnology,Denmark,479498.72,143.63
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,USA,354094.84,147.13
4,MRK,Merck & Co Inc,Healthcare,Drug Manufacturers - General,USA,333039.20,131.49
5,ABBV,Abbvie Inc,Healthcare,Drug Manufacturers - General,USA,301010.21,170.46
...,...,...,...,...,...,...,...
948,INDP,Indaptus Therapeutics Inc,Healthcare,Biotechnology,USA,20.84,2.44
949,IBIO,iBio Inc,Healthcare,Biotechnology,USA,20.69,2.40
952,BNTC,Benitec Biopharma Inc,Healthcare,Biotechnology,USA,20.16,7.41
953,CPIX,Cumberland Pharmaceuticals Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,20.15,1.42


In [4]:
df.to_csv('data/bio/tickers.csv', index=False)

In [5]:
client = RESTClient(api_key='AKADeecFYwcLpgjEt6rGFpQHbyHi7lBA')

In [10]:
news_df = pd.DataFrame(columns=['ticker', 'published', 'title', 'description', 'related_tickers'])
start_dt = (datetime.now() - timedelta(days=59)).replace(hour=0, minute=0, second=0, microsecond=0)
for ticker in tqdm(df['Ticker'].unique()):
    for news in client.list_ticker_news(ticker, published_utc_gte=start_dt.isoformat()):
        if not news.description or not news.title:
            continue
        if 'FDA' in news.description or 'FDA' in news.title:
            news_df = pd.concat([
                news_df,
                pd.DataFrame([[ticker, news.published_utc[:-1], news.title, news.description.replace('\n', ' '), ','.join(news.tickers)]], columns=news_df.columns)
            ], ignore_index=True)

news_df['published'] = pd.to_datetime(news_df['published']) - timedelta(hours=4)
news_df

100%|██████████| 670/670 [02:38<00:00,  4.22it/s]


,ticker,published,title,description,related_tickers
0,LLY,2024-06-11 14:19:10,Eli Lilly's Donanemab Gains Favorable FDA Advi...,BofA Securities analyst Geoff Meacham reiterat...,"BIIB,LLY,ESAIY"
1,LLY,2024-06-11 09:29:00,Eli Lilly (LLY) Alzheimer's Drug Donanemab Get...,"Eli Lilly (LLY) Alzheimer's disease drug, dona...","BIIB,LLY,PRTA"
2,LLY,2024-06-10 09:54:00,Biogen (BIIB) & Eisai's Leqembi sBLA Gets FDA ...,FDA accepts Biogen's (BIIB) partner Eisai's sB...,"BIIB,LLY,PRTA"
3,LLY,2024-05-15 12:12:00,Biogen (BIIB) Partner Begins Filing for Leqemb...,Biogen (BIIB) and Eisai start the rolling subm...,"BIIB,LLY,THRD"
4,LLY,2024-05-10 11:18:00,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,"PFE,MRK,LLY"
...,...,...,...,...,...
222,NERV,2024-06-11 14:06:00,"Regeneron (REGN), SNY Win FDA Nod for Kevzara ...",The FDA approves label expansion of Regeneron ...,"REGN,SNY,NERV,ALXO"
223,NERV,2024-06-10 09:42:00,AstraZeneca's (AZN) Tagrisso sNDA Gets FDA's P...,The FDA grants priority review to AstraZeneca'...,"AZN,NERV,MRNS,KRYS"
224,NERV,2024-05-23 12:12:00,Cartesian (RNAC) Up as FDA Grants RMAT Tag to ...,The FDA bestows RMAT designation to Cartesian'...,"NERV,MRNS,KRYS,RNAC"
225,ITRM,2024-05-31 08:00:00,Iterum Therapeutics Receives FDA Acceptance of...,Oral Sulopenem NDA has been assigned a PDUFA a...,ITRM


In [11]:
news_df.to_csv('data/bio/fda_news.csv', index=False)

In [12]:
unique_tickers = set(news_df.ticker.unique())
unique_tickers.update( set(pd.read_csv('data/bio/phase_news.csv').ticker.unique()) )
len(unique_tickers)

248

In [13]:
for ticker in tqdm(unique_tickers):
    cdf = yf.download(ticker, start=start_dt, end=datetime.now(), interval='2m', prepost=True, progress=False)
    cdf.drop(columns=['Adj Close']).to_sql(ticker, connection, if_exists='replace')

100%|██████████| 248/248 [02:00<00:00,  2.06it/s]
